In [1]:
!pip install torch transformers sentencepiece scikit-learn pandas

  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/1e/ce/7d251155a783fb2c1bb6837b2b7023c622a2070a0a72726ca1df47e7ea34/torch-2.9.1-cp311-none-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/6a/6b/2f416568b3c4c91c96e5a365d164f8a4a4a88030aa8ab4644181fdadce97/transformers-4.57.3-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.1 MB/s eta 0:00:00
  Obtaining dependency information for sentencepiece from https://files.pythonhosted.org/packages/bb/88/2b41e07bd24f33dcf2f18ec3b74247aa4af3526bad8907b8727ea3caba03/sentencepiece-0.2.1-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Using cached sentencepiece-0.2.1-cp311-cp311-macosx_11_0_arm64.whl.metadata (10 kB)
  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/18/d6/ba863a4171ac9d7314c4d3fc251f015704a2caeee41ced89f321c049ed83/scikit_learn-1.7.2-cp

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW  # 변경된 부분
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

# 1. 설정 및 하이퍼파라미터
MODEL_NAME = "monologg/kobert"  # Hugging Face에서 KoBERT 포팅 버전 사용
MAX_LEN = 64                    # 입력 텍스트 최대 길이 (진단명은 보통 짧으므로 64~128 충분)
BATCH_SIZE = 16
EPOCHS = 3
LEARNING_RATE = 2e-5
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"사용 장치: {DEVICE}")

# 2. 더미 데이터 생성 (실제 사용 시에는 CSV 등을 로드하세요)
# 예: 실제 데이터는 pd.read_csv('kcd_data.csv')
data = {
    'text': [
        '급성 비인두염', '감기 증상', '콧물과 기침', # J00
        '제2형 당뇨병', '성인 당뇨', '인슐린 비의존 당뇨병', # E11
        '상세불명의 위염', '급성 위염', '속쓰림 및 소화불량' # K29
    ],
    'label': ['J00', 'J00', 'J00', 'E11', 'E11', 'E11', 'K29', 'K29', 'K29']
}
df = pd.DataFrame(data)

# 라벨 인코딩 (KCD 코드 -> 숫자)
le = LabelEncoder()
df['encoded_label'] = le.fit_transform(df['label'])
NUM_LABELS = len(le.classes_)
print(f"분류할 클래스(KCD 코드) 개수: {NUM_LABELS}개 {le.classes_}")

# 3. 데이터셋 클래스 정의
class KCDDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# 4. 토크나이저 및 데이터로더 로드
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
dataset = KCDDataset(df['text'].values, df['encoded_label'].values, tokenizer, MAX_LEN)
data_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# 5. 모델 정의 (BERT for Sequence Classification)
model = BertForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=NUM_LABELS
)
model.to(DEVICE)

# 6. 학습 루프 (Training Loop)
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
loss_fn = nn.CrossEntropyLoss()

print("--- 학습 시작 ---")
model.train()
for epoch in range(EPOCHS):
    total_loss = 0
    for batch in data_loader:
        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)
        labels = batch['labels'].to(DEVICE)

        optimizer.zero_grad()

        # Hugging Face 모델은 내부적으로 loss를 계산하여 반환함
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    
    avg_loss = total_loss / len(data_loader)
    print(f"Epoch {epoch+1}/{EPOCHS} | Loss: {avg_loss:.4f}")

print("--- 학습 완료 ---")

# 7. 예측 함수 (Inference)
def predict_kcd_code(text):
    # 모델을 평가 모드로 전환
    model.eval() 

    # 2. 입력 텍스트 전처리 (토큰화)
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,      # 문장 시작([CLS])과 끝([SEP])에 특수 토큰 추가
        max_length=MAX_LEN,           # 설정한 최대 길이(64)에 맞춤
        return_token_type_ids=False,  # 문장이 하나이므로 세그먼트 ID는 불필요
        padding='max_length',         # 길이가 짧으면 나머지를 0(패딩)으로 채움
        truncation=True,              # 길이가 길면 잘라냄
        return_attention_mask=True,   # 패딩된 부분은 무시하도록 마스크 생성
        return_tensors='pt',          # 결과를 PyTorch 텐서(tensor) 형태로 반환
    )
    
    # 3. 입력 토큰을 모델에 입력
    input_ids = encoding['input_ids'].to(DEVICE)
    attention_mask = encoding['attention_mask'].to(DEVICE)

    # 4. 모델 예측
    with torch.no_grad(): # 예측 시에는 역전파(Backpropagation)가 필요 없으므로 기울기 계산을 꺼서 메모리를 아끼고 속도를 높입니다.
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs.logits, dim=1)
    
    # 5. 결과 변환 (숫자 -> 문자)
    predicted_label = le.inverse_transform(preds.cpu().data.numpy())[0]
    return predicted_label



사용 장치: cpu
분류할 클래스(KCD 코드) 개수: 3개 ['E11' 'J00' 'K29']


tokenizer_config.json:   0%|          | 0.00/263 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


--- 학습 시작 ---
Epoch 1/3 | Loss: 1.1077
Epoch 2/3 | Loss: 1.1328
Epoch 3/3 | Loss: 1.1018
--- 학습 완료 ---


In [ ]:
# 8. 테스트 실행
test_texts = [
    "환자가 콧물이 심하고 기침을 계속함",
    "혈당 수치가 높고 당뇨 관리가 필요함",
    "위가 쓰리고 소화가 잘 안됨",
    "빗길에 자빠짐"
]

print("\n--- 예측 결과 테스트 ---")
for text in test_texts:
    code = predict_kcd_code(text)
    print(f"입력: {text} -> 예측 코드: {code}")




--- 예측 결과 테스트 ---
입력: 환자가 콧물이 심하고 기침을 계속함 -> 예측 코드: K29
입력: 혈당 수치가 높고 당뇨 관리가 필요함 -> 예측 코드: K29
입력: 위가 쓰리고 소화가 잘 안됨 -> 예측 코드: K29
입력: 빗길에 자빠짐 -> 예측 코드: K29
